In [1]:
## Import necessary modules
import os,sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator, AutoLocator, FormatStrFormatter, ScalarFormatter
import numpy as np
import datetime, calendar
from datetime import timedelta
import matplotlib.patches as mpatches
from itertools import tee

%matplotlib tk

In [2]:
######## GET A LIST OF MATCHING .xls FILES FROM THE GIVEN DIRECTORY
arcpath='/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/'
def collectxlfiles(arcpath):
    xlfilelist=list()

    for xlfile in os.listdir(arcpath):
        if xlfile.startswith('1458_daily'):
            xlfileabs=os.path.join(arcpath,xlfile)
            xlfilelist.append(xlfileabs)
    return sorted(xlfilelist)

xlfilelist=collectxlfiles(arcpath)

FileNotFoundError: [Errno 2] No such file or directory: '/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/'

In [ ]:
def turn_xls_to_df(file):
    needecols=[0,1,2,3,4,5,12,21] # Liste der Spalten, die aus dem Excelfile gelesen werden sollen
    nucolnames=['tstp','clls','ange','verb','ht','tt','acw','lost'] # Namen, die spaeter auf die Spalten kommen
    
    excel_df=pd.read_excel(file,skiprows=3,skip_footer=1,usecols=needecols)  # die ersten 3 werden nicht benötigt, letzte auch nicht
    excel_df['Timestamp'] = pd.to_datetime(excel_df['Timestamp'], format=' %d.%m.%Y %H:%M ') # statt string soll das ein datetime werden
    excel_df.columns=nucolnames
    excel_df2=excel_df.set_index('tstp').copy() # die timestamps sollen der index sein
        
    return excel_df2

In [ ]:
greatframe=pd.DataFrame() # leeren df initialisieren
for i in xlfilelist:
    i_frame=turn_xls_to_df(i)
    greatframe=greatframe.append(i_frame) 
# alle files werden ordentlich in df konvertiert und an den ausserhalb der funktion kreierten df angehangen
# greatframe # hier sind alle daten vollständig enthalten, die gebraucht werden, der Ur-Frame

In [ ]:
uhrzeit=greatframe.index.time # nur die Uhrzeit der Timestamps im Index
wday=greatframe.index.weekday

# Nebenzeiten von 00:00-07:29 und wieder von 20:00-23:59
nulluhr=datetime.time(0, 0)
halbacht_am=datetime.time(7, 29)
siebzehn30=datetime.time(17, 30)
zwanzig=datetime.time(20, 0)
vierundzwanzig=datetime.time(23, 59)
dreizehn=datetime.time(13, 0)

WeekNZ = (wday.isin([0,1,2,3,4]) & ((uhrzeit >= nulluhr) & (uhrzeit <= halbacht_am) | ((uhrzeit >= siebzehn30) & (uhrzeit < vierundzwanzig))))
SamsNZ = (wday.isin([5]) & ((uhrzeit >= nulluhr) & (uhrzeit <= halbacht_am) | ((uhrzeit >= dreizehn) & (uhrzeit < vierundzwanzig))))
SonnNZ = (wday.isin([6]))

In [ ]:
Nebenzeit_CE = greatframe.loc[(WeekNZ|SamsNZ|SonnNZ)].copy() # Nebenzeit viertelstündlich

NZ_CE_DAY=Nebenzeit_CE.groupby(pd.TimeGrouper(freq='D')).sum().copy() # Nebenzeit nach Tagen gruppiert
NZ_CE_DAY['wd'] = NZ_CE_DAY.index.weekday

In [ ]:
index_month=NZ_CE_DAY.index.month # nur der Monat der Timestamps im Index
index_year=NZ_CE_DAY.index.year

q1 = ((index_month >= 1) & (index_month <= 3))   # Quartal 1 usw.
q2 = ((index_month >= 4) & (index_month <= 6))
q3 = ((index_month >= 7) & (index_month <= 9))
q4 = ((index_month >= 10) & (index_month <= 12))

In [ ]:
-

In [ ]:
jahr = 2017
Start_2017=datetime.date(2017,1,1)
EndQ1_2018=datetime.date(2018,3,31)
#Quart=NZ_CE_DAY.loc[(index_year == jahr) & q4].copy()
Quart=NZ_CE_DAY.loc[(NZ_CE_DAY.index.date >= Start_2017) & (NZ_CE_DAY.index.date <= EndQ1_2018)].copy()

plotzi(Quart,4)